In [5]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from darts import TimeSeries
from darts.models import TFTModel
from darts.metrics import mse, rmse, r2_score, mae, smape
from darts.dataprocessing.transformers import Scaler
import torch
from optuna.integration import PyTorchLightningPruningCallback
from pytorch_lightning.callbacks import Callback, EarlyStopping
import optuna
import os
import json

# Configurações de visualização
plt.style.use('ggplot')
plt.rcParams['figure.figsize'] = (12, 6)
torch.set_float32_matmul_precision('medium')

In [6]:
target_columns = [
    'Temperature','Precipitation_accumulated','Humidity', 'Wind_Speed_kmh',
    'Soil_Moisture', 'Soil_Temperature', 'Wind_Dir_Sin', 'Wind_Dir_Cos'
]

df = pd.read_csv("../data/ground_station_clean.csv")

In [7]:
encoders = {
    "cyclic":{
        "past":["month","dayofyear","day","hour","minute"],
        "future":["month","dayofyear","day","hour","minute"],
    },
    "transformer": Scaler()
}

In [8]:
class PatchedPruningCallback(optuna.integration.PyTorchLightningPruningCallback, Callback):
    pass
# Criar diretório para salvar os resultados das iterações
results_output_dir = "optuna_iteration_metrics"
os.makedirs(results_output_dir, exist_ok=True)

In [24]:
def objective(trial):

    output_chunk_length = trial.suggest_int("output_chunk_length", 12, 72)
    input_chunk_length = trial.suggest_int("input_chunk_length",output_chunk_length + 1, output_chunk_length + 72)
    hidden_size = trial.suggest_int('hidden_size', 32, 256)

    lstm_layers = trial.suggest_int('lstm_layers', 1, 2)
    num_attention_heads = trial.suggest_int('num_attention_heads', 2, 4)
    hidden_continuous_size = trial.suggest_int('hidden_continuous_size', 4, 32)
    add_relative_index_val = trial.suggest_categorical('add_relative_index', [False, True])

    dropout = trial.suggest_float('dropout', 0.0, 0.3, step=0.05)
    batch_size = trial.suggest_int("batch_size", 16, 64)
    lr = trial.suggest_float("lr", 1e-5, 1e-2, log=True)
    forecast_horizon = trial.suggest_int("forecast_horizon", 1, 24)
    

    pruner = PatchedPruningCallback(trial, monitor="val_loss")
    early_stopper = EarlyStopping("val_loss", min_delta=0.001, patience=10, verbose=True)
    pl_trainer_kwargs = {
        "accelerator": "auto",
        "callbacks": [pruner, early_stopper],
    }
    n = int(len(df)*0.8)
    train_df_fold, val_df_fold = df.iloc[:n], df.iloc[n:]
    train_fold = TimeSeries.from_dataframe(train_df_fold, time_col="Timestamp", value_cols=target_columns, freq='1h')
    val_fold = TimeSeries.from_dataframe(val_df_fold, time_col="Timestamp", value_cols=target_columns, freq='1h')

    print(f"\nIniciando Trial {trial.number}")
    print(f"Hiperparâmetros: {trial.params}")
    print("\nTreinando o modelo...")
    print(f"Train set: {len(train_fold)} samples") # Usar len() para TimeSeries
    print(f"Validation set: {len(val_fold)} samples") # Usar len() para TimeSeries
    
    scaler = Scaler()
    scaler = scaler.fit(train_fold)
    train_scaled = scaler.transform(train_fold)
    val_scaled = scaler.transform(val_fold)

    # Definir work_dir e model_name para consistência
    # force_reset=True irá limpar work_dir/model_name antes de cada treinamento da iteração.
    _work_dir = "/home/eduardo/Documentos/Water-Cycle-Neural-Network/darts_logs/" # Certifique-se que este caminho existe e é gravável
    _model_name = "model_optuna_temp" # Nome do modelo temporário para cada iteração

    # Criar o diretório _work_dir se não existir
    os.makedirs(_work_dir, exist_ok=True)

    model = TFTModel(
        model_name=_model_name,
        work_dir=_work_dir,
        input_chunk_length=input_chunk_length,
        output_chunk_length=output_chunk_length,
        hidden_size=hidden_size,
        n_epochs=30, # Considere tornar n_epochs um hiperparâmetro ou usar um valor maior com EarlyStopping
        batch_size=batch_size,
        lstm_layers=lstm_layers,
        num_attention_heads=num_attention_heads,
        hidden_continuous_size=hidden_continuous_size,
        add_relative_index=add_relative_index_val,
        dropout=dropout,
        add_encoders=encoders,
        pl_trainer_kwargs=pl_trainer_kwargs,
        loss_fn=torch.nn.HuberLoss(),
        lr_scheduler_cls=torch.optim.lr_scheduler.ReduceLROnPlateau,
        lr_scheduler_kwargs={"mode":"min", "factor":0.5, "patience":4, "min_lr":1e-6},
        save_checkpoints=True,    
        show_warnings=True,
        force_reset=True, # Importante para que cada iteração treine do zero com este model_name
        optimizer_kwargs={"lr": lr, "weight_decay": 1e-5},
    )
    model.fit(
        series=train_scaled,
        val_series=val_scaled,
        verbose=True,
        dataloader_kwargs={"num_workers": 11},
    )
    
    try:
        loaded_model = TFTModel.load_from_checkpoint(model_name=_model_name, work_dir=_work_dir, best=True)
        print(f"Modelo carregado do checkpoint para trial {trial.number}")
    except FileNotFoundError:
        print(f"Checkpoint não encontrado para {_model_name} em {_work_dir}. Usando o modelo treinado em memória.")
        loaded_model = model # Usar o modelo treinado se o carregamento falhar (fallback)


    forecasts = loaded_model.historical_forecasts(
        train_scaled,# Como no script original
        start=0.8, # Início da previsão no conjunto de treinamento
        forecast_horizon=forecast_horizon,
        stride=1, # Gera uma previsão em cada ponto de tempo possível
        retrain=False,
        verbose=True # Pode ser False
    ) 

    forecasts_t = scaler.inverse_transform(forecasts)
    s = scaler.inverse_transform(train_scaled).split_after(0.8)[1]
    metrics = {}
    print("Iniciando a verificação das séries temporais para constância...")
    try:
        for target in target_columns:
            metrics[target] = {
                'MSE': mse(s[target], forecasts_t[target]),
                'RMSE': rmse(s[target], forecasts_t[target]),
                'MAE': mae(s[target], forecasts_t[target]),
                'R2': r2_score(s[target], forecasts_t[target]),
                'SMAPE': smape(s[target], forecasts_t[target]),
            }
        metrics_df = pd.DataFrame(metrics).T
        print("\nMétricas de performance:")
        print(metrics_df)
    except Exception as e:
        print(e)

    overall_smape_val = smape(s, forecasts_t)
    print(f"O SMAPE desse fold foi {overall_smape_val}")

    trial_dict = {
        "trial_number": trial.number,
        "input_chunk_length": input_chunk_length,
        "output_chunk_length": output_chunk_length,
        "hidden_size": hidden_size,
        "lstm_layers": lstm_layers,
        "num_attention_heads": num_attention_heads,
        "hidden_continuous_size": hidden_continuous_size,
        "add_relative_index": add_relative_index_val,
        "dropout": dropout,
        "batch_size": batch_size,
        "lr": lr,
        "forecast_horizon": forecast_horizon,
        "overall_smape_val": overall_smape_val,
        "metrics": metrics_df.to_dict(),
    }
    json_path = os.path.join(results_output_dir, f"trial_{trial.number}.json")
    with open(json_path, 'w') as f:
        json.dump(trial_dict, f, indent=4)
    print(f"Resultados do trial {trial.number} salvos em {json_path}")
    return overall_smape_val if not np.isnan(overall_smape_val) else float("inf")



In [28]:
def print_callback(study, trial):
    print(f"Current value: {trial.value}, Current params: {trial.params}")
    print(f"Best value: {study.best_value}, Best params: {study.best_trial.params}")

In [27]:
study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=50, callbacks=[print_callback])

[I 2025-05-15 11:58:19,963] A new study created in memory with name: no-name-257c6297-1f70-422b-8380-7b823aa60ec9
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

   | Name                              | Type                             | Params | Mode 
------------------------------------------------------------------------------------------------
0  | criterion                         | HuberLoss                        | 0      | train
1  | train_criterion                   | HuberLoss                        | 0      | train
2  | val_criterion                     | HuberLoss                        | 0      | train
3  | train_metrics                     | MetricCollection                 | 0      | train
4  | val_metrics                       | MetricCollection                 | 0      | train
5  | input_embeddings                  | _MultiEmbedding                  | 0      |


Iniciando Trial 0
Hiperparâmetros: {'output_chunk_length': 24, 'input_chunk_length': 85, 'hidden_size': 96, 'lstm_layers': 1, 'num_attention_heads': 2, 'hidden_continuous_size': 23, 'add_relative_index': True, 'dropout': 0.15000000000000002, 'batch_size': 57, 'lr': 0.007635364276486364, 'forecast_horizon': 1}

Treinando o modelo...
Train set: 6647 samples
Validation set: 1662 samples


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

...

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

[I 2025-05-15 18:08:53,396] Trial 49 pruned. Trial was pruned at epoch 0.


Current value: 0.04513023639662728, Current params: {'output_chunk_length': 64, 'input_chunk_length': 122, 'hidden_size': 232, 'lstm_layers': 1, 'num_attention_heads': 4, 'hidden_continuous_size': 27, 'add_relative_index': True, 'dropout': 0.25, 'batch_size': 62, 'lr': 0.0002681880905869364, 'forecast_horizon': 8}
Best value: 51.62609841430928, Best params: {'output_chunk_length': 40, 'input_chunk_length': 60, 'hidden_size': 222, 'lstm_layers': 1, 'num_attention_heads': 4, 'hidden_continuous_size': 25, 'add_relative_index': True, 'dropout': 0.3, 'batch_size': 43, 'lr': 0.00031002620708540484, 'forecast_horizon': 7}


In [29]:
print(f"Melhor Valor (Mínimo): {study.best_value}")
print(f"Melhores Parâmetros: {study.best_params}")

Melhor Valor (Mínimo): 51.62609841430928
Melhores Parâmetros: {'output_chunk_length': 40, 'input_chunk_length': 60, 'hidden_size': 222, 'lstm_layers': 1, 'num_attention_heads': 4, 'hidden_continuous_size': 25, 'add_relative_index': True, 'dropout': 0.3, 'batch_size': 43, 'lr': 0.00031002620708540484, 'forecast_horizon': 7}


In [30]:
best_dict = {
    "best_value": study.best_value,
    "best_params": study.best_params,
}
json_path = os.path.join(results_output_dir, "best_trial.json")
with open(json_path, 'w') as f:
    json.dump(best_dict, f, indent=4)
print(f"Melhores resultados salvos em {json_path}")

Melhores resultados salvos em optuna_iteration_metrics/best_trial.json
